## Request NEXRAD Level-2 Radar Data.
This notebook is an example of how to use pydrology to download NEXRAD level-2 doppler radar data. NOAA provides access to NEXRAD doppler data across the contiguous United States. More information can be found here (https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00345). With this script, you can requeset raw NEXRAD Level-2 Base data and then further process it into a NetCDF4 file if desired. 

The general workflow for requesting data is as follows:
1. Choose a doppler radar site calling code from this map (https://www.roc.noaa.gov/WSR88D/Maps.aspx).
2. Download the data with a request to an output directory.
3. Process the reflectivity parameter into a NetCDF4 file if desired.

In [2]:
from pydrology.data_requests import request_nexrad_data, nexrad_to_netcdf
from pathlib import Path
import netCDF4 as nc4

## 1 & 2) Request/Download NEXRAD files.

In [3]:
site = 'KBGM'
date = '2022-07-25'
output_directory = Path.home() / 'Desktop' / 'NEXRAD_Output'

# Request and download NEXRAD data. Output is a list of file paths to downloaded NEXRAD files.
output_files = request_nexrad_data(site, date, output_directory)

Found 294 scans for this date... 

Downloaded KBGM20220725_000141_V06
1 out of 1 files downloaded...0 errors
[<LocalNexradFile object - /Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_000141_V06/KBGM20220725_000141_V06>]
/Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_000141_V06/KBGM20220725_000141_V06
Downloaded KBGM20220725_000711_V06
1 out of 1 files downloaded...0 errors
[<LocalNexradFile object - /Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_000711_V06/KBGM20220725_000711_V06>]
/Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_000711_V06/KBGM20220725_000711_V06
Downloaded KBGM20220725_001241_V06
1 out of 1 files downloaded...0 errors
[<LocalNexradFile object - /Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_001241_V06/KBGM20220725_001241_V06>]
/Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_001241_V06/KBGM20220725_001241_V06


## 3) Create NetCDF file from output data. 

In [4]:
# Create netCDF4 file from output files.
nc_path = nexrad_to_netcdf(output_files, output_directory, date, site)

# Inspect the netCDF4 file.
rootgrp = nc4.Dataset(nc_path, "r", format="NETCDF4")

print('\n', 'NETCDF INSPECTION')
print('\n', 'DIMENSIONS')
for key, val in rootgrp.dimensions.items():
    print(key, ':', val)
    
print('\n', 'VARIABLES')
for key, val in rootgrp.variables.items():
    print(key, ':', val)

rootgrp.close()

Processing file: /Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_000141_V06/KBGM20220725_000141_V06
Processing file: /Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_000711_V06/KBGM20220725_000711_V06
Processing file: /Users/alexyoung/Desktop/NEXRAD_Output/KBGM20220725_001241_V06/KBGM20220725_001241_V06
Created file: NEXRAD_KBGM_2022-07-25.nc

 NETCDF INSPECTION

 DIMENSIONS
time : <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 3
refLat : <class 'netCDF4._netCDF4.Dimension'>: name = 'refLat', size = 720
refLon : <class 'netCDF4._netCDF4.Dimension'>: name = 'refLon', size = 1832

 VARIABLES
time : <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: hours since 0001-01-01 00:00:00.0
    calendar: gregorian
unlimited dimensions: 
current shape = (3,)
filling on, default _FillValue of 9.969209968386869e+36 used
refLat : <class 'netCDF4._netCDF4.Variable'>
float32 refLat(refLat, refLon)
unlimited dimensions: 
current shape = (720, 1832)
filling on, default 